# RaritySniper Webscraper

Importing the required libraries

In [10]:
import requests
from requests_html import HTMLSession
import math
import csv
from bs4 import BeautifulSoup
import time

The module below establishes a connection with raritysniper, fetches the response, and calulcates the number of pages it will have to initially parse.

In [5]:
url = "https://search2.raritysniper.com/collections/collections/documents/search"


querystring = {"x-typesense-api-key":"L1NoMW9ITm1SYWNodFk4cWpmaHphQWZTS2tuaTVFWDNGdmxjT1llcEpLdz1uNWhMeyJmaWx0ZXJfYnkiOiJwdWJsaXNoZWQ6dHJ1ZSJ9","use_cache":"true","q":"","query_by":"searchName","highlight_fields":"name","facet_by":"","filter_by":"","facet_query":"","page":"1","per_page":"250","sort_by":"launchDate:desc"}
payload = ""
headers = {
    "authority": "search2.raritysniper.com",
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9,es;q=0.8",
    "if-modified-since": "Wed, 14 Dec 2022 08:40:56 GMT",
    "origin": "https://raritysniper.com",
    "referer": "https://raritysniper.com/",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "Windows",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}


r = requests.request("GET", url, data=payload, headers=headers, params=querystring)
data = r.json()
n_pages = math.ceil((data["found"])/250)

This module fetches all the collection titles and stores them into csv file called 'nftcollections'

In [6]:
mother_list =[["collection name", "url"]]
collection_names = []
collection_urls = []

for x in range (1,n_pages+1):
    querystring = {"x-typesense-api-key":"L1NoMW9ITm1SYWNodFk4cWpmaHphQWZTS2tuaTVFWDNGdmxjT1llcEpLdz1uNWhMeyJmaWx0ZXJfYnkiOiJwdWJsaXNoZWQ6dHJ1ZSJ9","use_cache":"true","q":"","query_by":"searchName","highlight_fields":"name","facet_by":"","filter_by":"","facet_query":"","page":f"{x}","per_page":"250","sort_by":"launchDate:desc"}
    r = requests.request("GET", url, data=payload, headers=headers, params=querystring)
    data = r.json()
    for i in range(0,(len(data['hits']))):
        baby_list = []
        baby_list.append((data['hits'][i]['document']['name']))
        raw_url = (data['hits'][i]['document']['collectionSlug'])
        collection_names.append(data['hits'][i]['document']['collectionSlug'])
        baby_list.append(("https://raritysniper.com/" + raw_url))
        collection_urls.append("https://raritysniper.com/" + raw_url)
        mother_list.append(baby_list)
        
with open("nftcollections.csv", "w", newline="", encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(mother_list)


The following module goes to each url as collected in the module above, and parses EACH nft from the colelctions. Caution - it can take upto 10-11 hours for this module to run. It stores the results in a csv file called 'cards'.

In [14]:
url = "https://search2.raritysniper.com/multi_search"
querystring = {"use_cache":"true","x-typesense-api-key":"L1NoMW9ITm1SYWNodFk4cWpmaHphQWZTS2tuaTVFWDNGdmxjT1llcEpLdz1uNWhMeyJmaWx0ZXJfYnkiOiJwdWJsaXNoZWQ6dHJ1ZSJ9"}
headers = {
    "authority": "search2.raritysniper.com",
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.9,es;q=0.8",
    "content-type": "text/plain",
    "origin": "https://raritysniper.com",
    "referer": "https://raritysniper.com/",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "Windows",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
}

mother_list = [['collection name', 'nft id', 'rank', 'rarity score']]

for x in collection_names:
    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)
    payload = "{\"searches\":[{\"collection\":\"assets_" + x + "\",\"q\":\"*\",\"max_facet_values\":44,\"page\":1,\"per_page\":250}]}"
    print("currently extracting cards from " + x + ".Time is:" + current_time)
    r = requests.request("POST", url, data=payload, headers=headers, params=querystring)
    data = r.json()
    n_hits = (data['results'][0]['found'])
    n_pages = math.ceil(n_hits/250)
    print("number of pages to parse = " + str(n_pages) + ".please wait.")
    for i in range(1, n_pages+1):
        payload = "{\"searches\":[{\"collection\":\"assets_" + x + "\",\"q\":\"*\",\"max_facet_values\":44,\"page\":" + str(i) + ",\"per_page\":250}]}"
        r = requests.request("POST", url, data=payload, headers=headers, params=querystring)
        data = r.json()
        n_hits = len((data['results'][0]['hits']))
        
        for j in range(0, n_hits):
            baby_list = []
            collection_name = (data['results'][0]['hits'][j]['document']['collectionSlug'])
            nft_id = ((data['results'][0]['hits'][j]['document']["nftId"]))
            rank = ((data['results'][0]['hits'][j]['document']["rank"]))
            r_score = ((data['results'][0]['hits'][j]['document']["rarityScore"]))
            baby_list.append(collection_name)
            baby_list.append(nft_id)
            baby_list.append(rank)
            baby_list.append(r_score)
            mother_list.append(baby_list)

with open("cards.csv", "w", newline="", encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(mother_list)
        

currently extracting cards from fable-dragons-by-tyrant.Time is:08:41:52
number of pages to parse = 3.please wait.
currently extracting cards from the-greedy-machines-vol-2.Time is:08:41:55
number of pages to parse = 9.please wait.
currently extracting cards from the-greedy-machines.Time is:08:42:02
number of pages to parse = 4.please wait.
currently extracting cards from cw-females.Time is:08:42:06
number of pages to parse = 27.please wait.
currently extracting cards from revellers.Time is:08:42:27
number of pages to parse = 8.please wait.
currently extracting cards from digi-miners.Time is:08:42:34
number of pages to parse = 32.please wait.
currently extracting cards from redefinedbulls.Time is:08:42:55
number of pages to parse = 7.please wait.
currently extracting cards from supercute-world.Time is:08:43:00
number of pages to parse = 23.please wait.
currently extracting cards from croswap-quartermasters.Time is:08:43:18
number of pages to parse = 13.please wait.
currently extracting

The last module below parses each collection and stores its social media links in a CSV file called "collection_socials". This takes a total of 2-3 hours to run.

In [15]:
mother_list = [['collection name','genie_url', 'opensea_url', 'looksrare_url', 'x2y2_url', 'rarible_url', 'discord_url', 'gem_url', 'twitter_url', 'ebisusbay_url', 'website_url']]
session = HTMLSession()
for url in collection_urls:
    
    baby_list = []
    collection_name = url.replace("https://raritysniper.com/",'')
    baby_list.append(collection_name)
    
    
    response = session.get(url, headers = headers)
    soup = BeautifulSoup(response.content, "html.parser")


    if (soup.find("img", {"alt" : "genie"})):
        genie = (soup.find("img", {"alt" : "genie"}).parent.get('href'))
    else:
        genie = ''

    baby_list.append(genie)

    if (soup.find("img", {"alt" : "opensea"})):
        opensea = (soup.find("img", {"alt" : "opensea"}).parent.get('href'))
    else:
        opensea = ''

    baby_list.append(opensea)

    if (soup.find("img", {"alt" : "looksrare"})):
        looksrare = (soup.find("img", {"alt" : "looksrare"}).parent.get('href'))
    else:
        looksrare = ''

    baby_list.append(looksrare)

    if (soup.find("img", {"alt" : "x2y2"})):
        x2y2 = (soup.find("img", {"alt" : "x2y2"}).parent.get('href'))
    else:
        x2y2 = ''

    baby_list.append(x2y2)

    if (soup.find("img", {"alt" : "rarible"})):
        rarible = (soup.find("img", {"alt" : "rarible"}).parent.get('href'))
    else:
        rarible = ''

    baby_list.append(rarible)

    if (soup.find("i", {"class" : "fab fa-discord hover:brightness-90 fa-lg text-2xl"})):
        discord = (soup.find("i", {"class" : "fab fa-discord hover:brightness-90 fa-lg text-2xl"}).parent.get('href'))
    else:
        discord = ''

    baby_list.append(discord)
    
    if (soup.find("img", {"alt" : "gem"})):
        gem = (soup.find("img", {"alt" : "gem"}).parent.get('href'))
    else:
        gem = ''

    baby_list.append(gem)

    if (soup.find("i", {"class" : "fab fa-twitter hover:brightness-90 fa-lg text-2xl"})):
        twitter = (soup.find("i", {"class" : "fab fa-twitter hover:brightness-90 fa-lg text-2xl"}).parent.get('href'))
    else:
        twitter = ''
    
    baby_list.append(twitter)
    
    
    if (soup.find("img", {"alt" : "ebisusbay"})):
        ebisusbay = (soup.find("img", {"alt" : "ebisusbay"}).parent.get('href'))
    else:
        ebisusbay = ''
    baby_list.append(ebisusbay)
    
    
    if (soup.find("img", {"alt" : "website"})):
        website = (soup.find("img", {"alt" : "website"}).parent.get('href'))
    else:
        website = ''
    baby_list.append(website)
    mother_list.append(baby_list)

with open("collection_socials.csv", "w", newline="", encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(mother_list)